In [47]:
from abc import ABC, abstractmethod
import re
from sacremoses import MosesTokenizer
import Levenshtein
import spacy
import nltk
import pickle
import urllib
import os
import tarfile
import zipfile
import seaborn as sns
import pandas as pd
from tqdm import tqdm
from pathlib import Path
import numpy as np
# import paths
from string import punctuation
import matplotlib.pyplot as plt
nltk.download('stopwords', quiet=True)
from nltk.corpus import stopwords
from nltk import word_tokenize
RESOURCES_DIR = Path('../resources')
DATASETS_PATH = RESOURCES_DIR / "datasets"
WORD_EMBEDDINGS_NAME ='cc.nl.300'
DUMPS_DIR = RESOURCES_DIR / "DUMPS"

import gensim
stopwords = set(stopwords.words("dutch"))

In [48]:
def ControlDivisionByZero(numerator, denominator):
    return numerator / denominator if denominator != 0 else 0


class FeatureAbstract(ABC):

    @abstractmethod
    def get_ratio(self, kwargs):
        pass

    @abstractmethod
    def calculate_ratio(self, simple_text, original_text):
        pass


class Feature(FeatureAbstract):

    def __init__(self, split, target_ratio):
        self.split = split
        self.target_ratio = target_ratio

    def get_ratio(self, kwargs):
        if not 'original_text_preprocessed' in kwargs:
            kwargs['original_text_preprocessed'] = ""

        if self.split == "train":
            simple_text = kwargs.get('simple_text')
            original_text = kwargs.get('original_text')
            result_ratio = self.calculate_ratio(simple_text, original_text)

        elif self.split == "valid" or self.split == "test":
            result_ratio = self.target_ratio
        else:
            raise ValueError("stage value not supported")
        kwargs['original_text_preprocessed'] += f'{self.name}_{result_ratio} '
        return kwargs

    @property
    def name(self):
        class_name = self.__class__.__name__
        name = ""
        for word in re.findall('[A-Z][^A-Z]*', class_name):
            if word: name += word[0]
        if not name: name = class_name
        return name


class WordLengthRatio(Feature):

    def __init__(self, stage, target_ratio):
        super().__init__(stage, target_ratio)
        if stage == "train":
            # THIS IS A WORD TOKENIZER, we need one for dutch
            # nl_core_news_sm spacy  spacy.nl_core_news_sm 
            # nltk.word_tokenize
            self.tokenizer =  MosesTokenizer(lang='nl') #  nltk.word_tokenize(language='dutch')  # Moses Tokenizer for Dutch language

    def calculate_ratio(self, simple_text, original_text):
        return round(ControlDivisionByZero(
            len(self.tokenizer.tokenize(simple_text)),
            len(self.tokenizer.tokenize(original_text))), 2)


class CharLengthRatio(Feature):

    def __init__(self, stage, target_ratio):
        super().__init__(stage, target_ratio)

    def calculate_ratio(self, simple_text, original_text):
        return round(ControlDivisionByZero(len(simple_text),
            len(original_text)), 2)


class LevenshteinRatio(Feature):

    def __init__(self, stage, target_ratio):
        super().__init__(stage, target_ratio)

    def calculate_ratio(self, simple_text, original_text):
        simple_text = word_tokenize(simple_text,language='dutch')
        original_text = word_tokenize(original_text,language='dutch')
        # complex_sentence = tokenize(complex_sentence) # ,language='dutch')

        # simple_sentence = tokenize(simple_sentence) # ,language='dutch')
        return round(Levenshtein.ratio(original_text,
                                       simple_text), 2)


class DependencyTreeDepthRatio(Feature):

    def __init__(self, stage, target_ratio):
        super().__init__(stage, target_ratio)
        if stage == "train":
            self.nlp = self.get_spacy_model()

    def get_spacy_model(self):

        model = 'nl_core_news_sm'  # from spacy, Dutch pipeline optimized for CPU. Components: tok2vec, morphologizer, tagger, parser, lemmatizer (trainable_lemmatizer), senter, ner.
        if not spacy.util.is_package(model):
            spacy.cli.download(model)
            spacy.cli.link(model, model, force=True, model_path=spacy.util.get_package_path(model))
        return spacy.load(model)

    def calculate_ratio(self, simple_text, original_text):

        result_ratio = round(ControlDivisionByZero(
            self.get_dependency_tree_depth(simple_text),
            self.get_dependency_tree_depth(original_text)), 2)

        return result_ratio

    def get_dependency_tree_depth(self, sentence):

        def get_subtree_depth(node):
            if len(list(node.children)) == 0:
                return 0
            return 1 + max([get_subtree_depth(child) for child in node.children])

        tree_depths = [get_subtree_depth(spacy_sentence.root) for spacy_sentence in self.nlp(sentence).sents]
        if len(tree_depths) == 0:
            return 0
        return max(tree_depths)


class WordRankRatio(Feature):
    # single underscore = internally

    def __init__(self, stage, target_ratio): # constructor of the class 
        super().__init__(stage, target_ratio)
        if stage == "train":
            self.tokenizer = MosesTokenizer(lang='nl')
            self.word2rank = self._get_word2rank()
            print('finished get word2rank')
            # store in file
            # if not present, make file
            # with open ("./resources/DUMPS/word2rank.txt", "a", encoding="utf8") as file: 
            #     file.writelines(self.word2rank)
            #     file.write("\n")
            #     file.close()
            print('length of word2rank', len(self.word2rank))
            self.length_rank = len(self.word2rank) # hier length of the file! 

    def calculate_ratio(self, simple_text, original_text):

        result_ratio = round(min(ControlDivisionByZero(self.get_lexical_complexity_score(simple_text),
                                                       self.get_lexical_complexity_score(original_text)),
                                 2), 2)

        return result_ratio

    def get_lexical_complexity_score(self, sentence, quantile_value=0.75):

        words = self.tokenizer.tokenize(self._remove_stopwords(self._remove_punctuation(sentence)))
        words = [word for word in words if word in self.word2rank]
        if len(words) == 0:
            return np.log(1 + self.length_rank)
        return np.quantile([self._get_rank(word) for word in words], quantile_value)

    def _remove_punctuation(self, text):
        return ' '.join([word for word in self.tokenizer.tokenize(text) if not self._is_punctuation(word)])

    def _remove_stopwords(self, text):
        return ' '.join([w for w in self.tokenizer.tokenize(text) if w.lower() not in stopwords])

    def _is_punctuation(self, word):
        return ''.join([char for char in word if char not in punctuation]) == ''

    def _get_rank(self, word):
        rank = self.word2rank.get(word, self.length_rank)
        return np.log(1 + rank)

    def _get_word2rank(self, vocab_size=np.inf):
        model_filepath = DUMPS_DIR / f"{WORD_EMBEDDINGS_NAME}.pk"
        if model_filepath.exists():
            with open(model_filepath, 'rb') as f:
                model = pickle.load(f)
            return model
        else:
            print("Opening conll model...")
            print("Preprocessing word2rank...")
            DUMPS_DIR.mkdir(parents=True, exist_ok=True)
            WORD_EMBEDDINGS_PATH = DUMPS_DIR / f'{WORD_EMBEDDINGS_NAME}.txt'
            lines_generator = self._yield_lines(WORD_EMBEDDINGS_PATH)
            word2rank = {}
            # next(lines_generator)
            for i, line in enumerate(lines_generator):
                if i >= vocab_size: break
                word = line.split(' ')[0]
                word2rank[word] = i

            pickle.dump(word2rank, open(model_filepath, 'wb'))
            # txt_file = DUMPS_DIR / f'{WORD_EMBEDDINGS_NAME}.txt'
            # zip_file = DUMPS_DIR / f'{WORD_EMBEDDINGS_NAME}.zip'
            # if txt_file.exists(): txt_file.unlink()
            # if zip_file.exists(): zip_file.unlink()
            return word2rank
        
        # else:            
        #     print("Downloading dutch embeddings ...") # pretrained vectors
        #     self._download_twitter_embeddings(model_name='coostco', dest_dir=str(DUMPS_DIR))
        #     print("Preprocessing word2rank...")
        #     DUMPS_DIR.mkdir(parents=True, exist_ok=True)
        #     WORD_EMBEDDINGS_PATH = DUMPS_DIR / f'{WORD_EMBEDDINGS_NAME}.bin'
        #     model = self._load_word_embeddings(WORD_EMBEDDINGS_PATH) # returns index_to_key
        #     # store into file
        #     lines_generator = model # self._yield_lines(model) # (WORD_EMBEDDINGS_PATH)
            
        #     word2rank = {}
        #     # next(lines_generator)
        #     print('vocab_size', vocab_size)
        #     for i, line in enumerate(lines_generator):
        #         if i >= vocab_size: break # its not vocab size any more but  # len(model.key_to_index)
        #         word = line.split(',')[0]
        #         print('word', word)
        #         word2rank[word] = i
        #         print('ranked word?', word2rank[word])
                
        #     pickle.dump(word2rank, open(model_filepath, 'wb'))
        #     txt_file = DUMPS_DIR / f'{WORD_EMBEDDINGS_NAME}.txt'
        #     zip_file = DUMPS_DIR / f'{WORD_EMBEDDINGS_NAME}.zip'
        #     # if txt_file.exists(): txt_file.unlink()
        #     # if zip_file.exists(): zip_file.unlink()
        #     # print(word2rank)
        #     return word2rank
    
    def _load_word_embeddings(self, filepath):
        model = gensim.models.KeyedVectors.load_word2vec_format(filepath, binary=True) # '../resources/DUMPS/model.bin'
        model_indexes = model.index_to_key
        return model_indexes
    
    def _download_twitter_embeddings(self, model_name, dest_dir): # pretrained rankings
        url = ''
        if model_name == 'coosto_model':
            url = 'https://github.com/coosto/dutch-word-embeddings/releases/download/v1.0/model.bin'
        file_path = self._download_url(url, dest_dir)
        out_filepath = Path(file_path)
        out_filepath = out_filepath.parent / f'{out_filepath.stem}.txt'
        # print(out_filepath, out_filepath.exists())
        if not out_filepath.exists():
            print("Extracting: ", Path(file_path).name)
            self._unzip(file_path, dest_dir)

    def _yield_lines(self, filepath):
        filepath = Path(filepath)
        with filepath.open('r', encoding="latin-1") as f:
            for line in f:
                # print(line)
                yield line.rstrip()

    def _download_url(self, url, output_path):
        name = url.split('/')[-1]
        file_path = f'{output_path}/{name}'
        if not Path(file_path).exists():
            with tqdm(unit='B', unit_scale=True, leave=True, miniters=1,
                      desc=name) as t:  # all optional kwargs
                urllib.request.urlretrieve(url, filename=file_path, reporthook=self._download_report_hook(t), data=None)
        return file_path

    def _unzip(self, file_path, dest_dir=None):
        if dest_dir is None:
            dest_dir = os.path.dirname(file_path)
        if file_path.endswith('.zip'):
            with zipfile.ZipFile(file_path, "r") as zip_ref:
                zip_ref.extractall(dest_dir)
        elif file_path.endswith("tar.gz") or file_path.endswith("tgz"):
            tar = tarfile.open(file_path, "r:gz")
            tar.extractall(dest_dir)
            tar.close()
        elif file_path.endswith("tar"):
            tar = tarfile.open(file_path, "r:")
            tar.extractall(dest_dir)
            tar.close()

    def _download_report_hook(self, t):
        last_b = [0]

        def inner(b=1, bsize=1, tsize=None):
            if tsize is not None:
                t.total = tsize
            t.update((b - last_b[0]) * bsize)
            last_b[0] = b

        return inner

In [49]:
complex_sentence = "Sommige steden aan de Eyre Highway in de zuidoostelijke hoek van West-Australië, tussen de Zuid-Australische grens bijna tot aan Caiguna, volgen de officiële West-Australische tijd niet."
simple_sentence = "Sommige steden in West-Australië gebruiken geen West-Australische tijd."

In [50]:
wordRank = WordRankRatio("train", 0.8)

finished get word2rank
length of word2rank 2000000


In [51]:
wordRank.get_lexical_complexity_score(complex_sentence)

9.982252521171782

In [52]:
wordRank.get_lexical_complexity_score(simple_sentence)

7.6787889981991535

In [53]:
complex_sentence = "Het ruimtevaartuig bestaat uit twee hoofdelementen: de NASA Cassini-orbiter, genoemd naar de Italiaans-Franse astronoom Giovanni Domenico Cassini, en de ESA Huygens-sonde, genoemd naar de Nederlandse astronoom, wiskundige en natuurkundige Christiaan Huygens."
simple_sentence = "Het ruimtevaartuig heeft twee hoofdelementen: de NASA Cassini-orbiter en de ESA Huygens-sonde."

In [54]:
wordRank.get_lexical_complexity_score(complex_sentence)

10.212353279947028

In [55]:
wordRank.get_lexical_complexity_score(simple_sentence)

12.142957986385708

In [56]:
complex_sentence = "Dit was het gebied ten oosten van de monding van de rivier de Vistula, later ook wel 'eigenlijk Pruisen' genoemd."
simple_sentence="Pruisen was eigenlijk de plaats ten oosten van de monding van de rivier de Vistula."

In [57]:
wordRank.get_lexical_complexity_score(complex_sentence)

8.799416316922343

In [58]:
wordRank.get_lexical_complexity_score(simple_sentence)

9.029323643692715

In [59]:
model = gensim.models.KeyedVectors.load_word2vec_format('../resources/DUMPS/cc.nl.300.txt' , binary=False)

In [60]:
model.most_similar("monding")

[('uitmonding', 0.6713468432426453),
 ('mondingen', 0.6680971384048462),
 ('mondingsgebied', 0.6594777703285217),
 ('rivier', 0.6502463221549988),
 ('Monding', 0.63310307264328),
 ('riviermonding', 0.621979296207428),
 ('oever', 0.6173555850982666),
 ('inmonding', 0.6017937660217285),
 ('bovenloop', 0.5896535515785217),
 ('benedenloop', 0.5862765908241272)]

In [61]:
model.most_similar("schiereiland")

[('schiereilandje', 0.7253426909446716),
 ('schiereilanden', 0.7160058617591858),
 ('eiland', 0.6915854215621948),
 ('Schiereiland', 0.6885219216346741),
 ('schiereilandjes', 0.6633923053741455),
 ('eilandje', 0.6587224006652832),
 ('Sinaïschiereiland', 0.6569958925247192),
 ('Sinaï-schiereiland', 0.6565256118774414),
 ('Gydan-schiereiland', 0.6527586579322815),
 ('Taz-schiereiland', 0.6515459418296814)]

In [62]:
model.most_similar("chitine") # what to do with that? 

[('chitineus', 0.7659825086593628),
 ('chitinepantser', 0.7478009462356567),
 ('chitinelaag', 0.7477145195007324),
 ('Chitine', 0.7288618683815002),
 ('chitineuze', 0.7251715660095215),
 ('chitinase', 0.6549855470657349),
 ('chitosan', 0.5534986853599548),
 ('quercitine', 0.5291063785552979),
 ('Ipakitine', 0.5217061042785645),
 ('adiponectine', 0.519142746925354)]

In [63]:
model.most_similar("muiterij")

[('muiterijen', 0.7211874723434448),
 ('muiten', 0.6396942734718323),
 ('muiter', 0.6047860383987427),
 ('muiters', 0.6009116172790527),
 ('muitend', 0.5844631195068359),
 ('muitende', 0.5715579986572266),
 ('Muiterij', 0.5705426335334778),
 ('opstand', 0.5567310452461243),
 ('muitelingen', 0.509614109992981),
 ('opstandelingen', 0.501315712928772)]

In [64]:
model.most_similar("moslimminderheid")

[('moslimminderheden', 0.873995304107666),
 ('moslimmeerderheid', 0.7240265011787415),
 ('moslimmigranten', 0.7145824432373047),
 ('moslimmilities', 0.6938619017601013),
 ('moslimpopulatie', 0.664075493812561),
 ('moslimmilitanten', 0.6638409495353699),
 ('moslim-meerderheid', 0.6448001861572266),
 ('moslimimmigranten', 0.626603901386261),
 ('moslim-immigranten', 0.6241893768310547),
 ('moslimachtergrond', 0.6201066374778748)]

In [65]:
model.most_similar("vaginale") 

[('Vaginale', 0.7293235659599304),
 ('vulvovaginale', 0.712486743927002),
 ('vaginal', 0.695442795753479),
 ('transvaginale', 0.6486552953720093),
 ('vaginaal', 0.6286012530326843),
 ('vagina', 0.6207197904586792),
 ('vaginaopening', 0.6058916449546814),
 ('vaginawand', 0.5986759066581726),
 ('urethrale', 0.5705299973487854),
 ('genitale', 0.5629680752754211)]

In [66]:
model.most_similar("tandheelkundige")

[('tandheelkunde', 0.7470686435699463),
 ('Tandheelkundige', 0.7367693185806274),
 ('tandheelkundigen', 0.733983039855957),
 ('tandheelkundig', 0.7016889452934265),
 ('orthodontische', 0.6754381656646729),
 ('tandtechnische', 0.6710221171379089),
 ('prothetische', 0.6651920080184937),
 ('medische', 0.6603924632072449),
 ('tandprothetische', 0.658509373664856),
 ('paramedische', 0.6571154594421387)]

In [67]:
model.key_to_index["impasse"]

43204

In [68]:
model.key_to_index["gitaarsnaar"]

KeyError: 'gitaarsnaar'

In [ ]:
model.key_to_index["aardnoten"]

370665

In [ ]:
model.key_to_index["van"]

4

In [ ]:
model.most_similar("Warmtekrachtkoppeling")

[('warmtekrachtkoppeling', 0.8105178475379944),
 ('Warmte-krachtkoppeling', 0.7976646423339844),
 ('Warmtekracht', 0.7961313128471375),
 ('micro-warmtekrachtkoppeling', 0.7938953042030334),
 ('warmtekrachtkoppelingen', 0.7838013768196106),
 ('microwarmtekrachtkoppeling', 0.7728864550590515),
 ('warmtekrachtkoppelingsinstallatie', 0.7693067789077759),
 ('Warmtekrachtinstallatie', 0.7620224356651306),
 ('warmtekrachtkoppelingsinstallaties', 0.746272623538971),
 ('warmte-krachtkoppeling', 0.6955339908599854)]

In [ ]:
model.most_similar("van")

[('het', 0.6326913237571716),
 ('voor', 0.6319442987442017),
 ('bij', 0.6279339790344238),
 ('daarvan', 0.6101812720298767),
 ('met', 0.604983389377594),
 ('hiervan', 0.603034496307373),
 ('rondom', 0.5895820260047913),
 ('Van', 0.5798993706703186),
 ('ervan', 0.5730587840080261),
 ('daarvan.De', 0.570547878742218)]